# Import

In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [2]:
# 14s
metadata = pd.read_json(directory + 'metadata.json', lines=True)
magazine = pd.read_json(directory + 'magazine.json', lines=True)
users = pd.read_json(directory + 'users.json', lines=True)

# Article_csv Load

In [3]:
# Read data
total_top_50000 = pd.read_csv(directory+'total_top_50000.csv')
total_top_50000 = total_top_50000.drop('Unnamed: 0',1)
recent_top_5000 = pd.read_csv(directory+'recent_top_5000.csv')
recent_top_5000 = recent_top_5000.drop('Unnamed: 0',1)
article_20190222_20190301 = pd.read_csv(directory+'article_20190222_20190301.csv')
article_20190222_20190301 = article_20190222_20190301.drop('Unnamed: 0',1)
article_20190301_20190314 = pd.read_csv(directory+'article_20190301_20190314.csv')
article_20190301_20190314 = article_20190301_20190314.drop('Unnamed: 0',1)

article_20190222_20190314 = article_20190222_20190301.append(article_20190301_20190314)
article_20190222_20190314['reg_datetime']  = article_20190222_20190314['reg_datetime'].map(lambda x : pd.to_datetime(x,format='%Y-%m-%d'))
# 20190314 보다 큰 건 제외
a = 20190314
a = pd.to_datetime(a,format='%Y%m%d')
article_20190222_20190314 = article_20190222_20190314[article_20190222_20190314['reg_datetime'] <= a]
article_20190222_20190314 = article_20190222_20190314.sort_values('reg_datetime')
len(article_20190222_20190314)


D:\ANACONDA\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


15165

In [4]:
import ast
popular_20190201_20190301_10 = pd.read_csv(directory+'popular_20190201_20190301_10.csv')
popular_20190201_20190301_10 = popular_20190201_20190301_10.drop('Unnamed: 0',1)
popular_20190201_20190301_10[:1]
popular_20190201_20190301_10['reg_datetime']  = popular_20190201_20190301_10['reg_datetime'].map(lambda x : pd.to_datetime(x,format='%Y-%m-%d'))
popular_20190201_20190301_10['keyword_list']  = popular_20190201_20190301_10['keyword_list'].map(lambda x : ast.literal_eval(x))

In [5]:
article_total_matrix_1 = np.load(directory+'article_total_matrix.npy')
article_20190222_20190314_matrix_1 = np.load(directory+'article_20190222_20190314_matrix.npy')
popular_20190201_20190301_10_matrix_1 = np.load(directory+'popular_20190201_20190301_10_matrix.npy')

In [6]:
print(article_total_matrix_1.shape)
print(article_20190222_20190314_matrix_1.shape)
print(popular_20190201_20190301_10_matrix_1.shape)

(643104, 128)
(15165, 128)
(55734, 128)


# User_df load

In [7]:
dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    #print(len(lines))
    #print(type(lines))
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')

In [8]:
import ast

user_df = pd.read_csv(directory+'user_df.csv')
user_df = user_df.drop('Unnamed: 0',1)
user_df['read_articles'] = user_df['read_articles'].map(lambda x : ast.literal_eval(x))
print(len(user_df))

3000


In [9]:
from tqdm import tqdm
import random

user_id = user_df['id'].tolist()
user_visit_cnt = user_df['visit_cnt'].tolist()
user_read_cnt = user_df['read_cnt'].tolist()
user_read_articles = user_df['read_articles'].tolist()
user_isNew =[]
for i in range(len(user_id)):
    if(user_read_cnt[i] <= 5):
        user_isNew.append(1)
    else:
        user_isNew.append(0)

# 1:34 ~ 1:35
user_following_list = []
for i in range(len(user_id)):
    li =list(users[users['id'] == user_id[i]]['following_list'])
    if(len(li) != 0):
        user_following_list.append(li[0])
    else:
        user_following_list.append([])

In [10]:
# 2mins
following_article_list = []
for i in tqdm(range(3000),desc='progress',mininterval=20):
    fl = user_following_list[i]
    if(len(fl) > 200):
        fl = random.choices(fl,k=200)
    if(len(fl) != 0):
        following_articles = []
        for author in fl:
            following_articles += popular_20190201_20190301_10[popular_20190201_20190301_10['author_id'] == author]['article_id'].tolist()
        following_article_list.append(following_articles)
    else :
        following_article_list.append([])

progress: 100%|████████████████████████████████████████████████████████████████████| 3000/3000 [05:51<00:00,  8.54it/s]


In [ ]:
popular_20190201_20190301_10

In [11]:
user_matrix_1 = np.load(directory+'user_matrix.npy')
print(type(user_matrix_1))
print(user_matrix_1.shape)
print(user_matrix_1.dtype)

<class 'numpy.ndarray'>
(3000, 128)
float32


# Recommend

In [12]:
def cos_similarity(x,y,eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2))+eps)
    ny = y / (np.sqrt(np.sum(y**2))+eps)
    return np.dot(nx,ny)

In [13]:
def recommend_noraml_algo(idx, num_total, num_recent, num_following, num_keyword):
    
    recommend_total = []
    if(num_total > 0):
        cnt = 0
        i = 0
        while(cnt < 100):
            artic = list(total_top_50000[i:i+1]['article_id'])[0]
            if artic not in user_read_articles[idx]:
                recommend_total.append(artic)
                cnt+=1
            i+= 1
            if(i>50000):
                break
    
    recommend_recent = []
    if(num_recent > 0):
        cnt = 0
        i = 0
        while(cnt < 100):
            artic = list(popular_20190201_20190301_10[i:i+1]['article_id'])[0]
            if artic not in user_read_articles[idx]:
                recommend_recent.append(artic)
                cnt+=1
            i +=1
            if(i >5000):
                break
        
    recommend_following = []
    if(num_following > 0):
        following_cnt = 0
        article_list = following_article_list[idx]
        if(len(article_list) != 0):
            random.shuffle(article_list)
            for artic in article_list:
                if artic not in user_read_articles[idx]:
                    recommend_following.append(artic)
                    following_cnt+=1
                if(following_cnt > 100):
                    break
    
    recommend_keyword = []
    if(num_keyword > 0):
        user_vector_x = user_matrix_1[idx]
        cos_sim = []

        for i in range(len(popular_20190201_20190301_10_matrix_1)):
            cos_sim.append(cos_similarity(user_vector_x,popular_20190201_20190301_10_matrix_1[i]))
        top_k = 1000
        cos_s = np.asarray(cos_sim)
        cos_sim = cos_s.argsort()[-top_k -1 : -1][::-1].tolist()
        cnt = 0
        for i in range(1000):
            artic = list(popular_20190201_20190301_10[cos_sim[i]:cos_sim[i]+1]['article_id'])[0]
            if artic not in user_read_articles[idx]: 
                recommend_keyword.append(list(popular_20190201_20190301_10[cos_sim[i]:cos_sim[i]+1]['article_id'])[0])
                cnt+=1
            if cnt >= 100 :
                break
    
    return recommend_total, recommend_recent, recommend_following, recommend_keyword
    

In [14]:
recommend_total, recommend_recent, recommend_following, recommend_keyword = recommend_noraml_algo(0,30,30,40,1)

In [24]:
recommend_new = ''
for x in recommend_recent:
    recommend_new += ' ' + x

In [ ]:
recommend_new

# Ratio 변경하면서 결과 비교
- ratio1 : 각각 성능이 얼마나 나오는지 비교해보기
- ratio2 
- ratio3 

In [ ]:
ratio1 =[[0,100,0,0],[0,0,0,100],[50,50,0,0],[10,50,0,40]]
# hyperparameter
for ratio in ratio1:
    num_total = ratio[0]
    num_recent = ratio[1]
    num_following = ratio[2]
    num_keyword = ratio[3]
    hyp = str(num_total)+'_'+str(num_recent)+'_' + str(num_following)+'_' + str(num_keyword)
    try:
        if not(os.path.isdir(directory + hyp )):
            os.makedirs(os.path.join(directory + hyp))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

    with open(directory + hyp +'/recommend' + '.txt','w') as file:
        for i in tqdm(range(len(user_df)),mininterval=20):
            uid = dev_users_list[i]
            i = user_id.index(uid)
            uid = user_id[i]
            isNew = user_isNew[i]
            if(isNew == 1):
            # 신규 회원
                file.write(uid + recommend_new)
                file.write('\n')
            else:
            # 기존 회원
                # 함수
                # list로 반환
                # hyperparmeter 비율 정해서 반환 (str 형태로)
                recommend_total, recommend_recent, recommend_following, recommend_keyword = recommend_noraml_algo(i,num_total,num_recent,num_following,num_keyword)
                recommend_total = recommend_total[:num_total]
                recommend_recent = recommend_recent[:num_recent]
                if( num_following > len(recommend_following)):
                    num_following = len(recommend_following)
                recommend_following = recommend_following[:num_following]
                recommend_keyword = recommend_keyword[:num_keyword]

                # 골고루 합치기 
                f_i=0
                k_i=0
                recommend_f_k = []
                for cnt_fk in range(num_following + num_keyword):
                    if f_i < num_following:
                        recommend_f_k.append(recommend_following[f_i])
                        f_i+=1
                    if k_i < num_keyword:
                        recommend_f_k.append(recommend_keyword[k_i])
                        k_i+=1

                recommend_r_t = []
                r_i = 0
                t_i = 0
                for cnt_rt in range(num_recent+num_total):
                    if r_i < num_recent:
                        recommend_r_t.append(recommend_recent[r_i])
                        r_i += 1

                    if t_i < num_total:
                        recommend_r_t.append(recommend_total[t_i])
                        t_i += 1

                cnt_rt = len(recommend_r_t)
                rt_i = 0
                cnt_fk = len(recommend_f_k)
                fk_i = 0
                recommend_all = []
                for cnt in range(cnt_fk + cnt_rt):
                    if(rt_i < cnt_rt):
                        recommend_all.append(recommend_r_t[rt_i])
                        rt_i +=1
                    if(fk_i < cnt_fk):
                        recommend_all.append(recommend_f_k[fk_i])
                        fk_i +=1
                recommend_all = set(recommend_all)
                # 부족한 경우 recent 에서
                if(len(recommend_all) < 100):
                    ar_id = 0
                    while(len(recommend_all) != 100):
                        artic = list(popular_20190201_20190301_10[ar_id:ar_id+1]['article_id'])[0]
                        if artic not in user_read_articles[i]:
                            recommend_all.add(artic)
                        ar_id +=1
                # 들어가는거 중복 제거 안해준것 같은데...
                
                recommend_all = list(recommend_all)
                # 마지막 최종 output
                recommend_normal = ''
                for x in recommend_all:
                    recommend_normal += ' '+ x

                file.write(uid + recommend_normal)
                file.write('\n')

 18%|█████████████▌                                                               | 528/3000 [12:27<1:01:49,  1.50s/it]

# Similarity 계산

- user_matrix_1 (3000,128)
- article_20190222_20190314_matrix_1 (15---,128)

In [31]:
def cos_similarity(x,y,eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2))+eps)
    ny = y / (np.sqrt(np.sum(y**2))+eps)
    return np.dot(nx,ny)

In [ ]:
tqdm(range(3000),desc='progress',mininterval=20)

In [36]:
user_vector_x = user_matrix_1[0]
cos_sim = []
    
for i in tqdm(range(len(article_total_matrix_1)),mininterval=10):
    cos_sim.append(cos_similarity(user_vector_x,article_total_matrix_1[i]))


100%|███████████████████████████████████████████████████████████████████████| 643104/643104 [00:17<00:00, 36331.67it/s]


In [37]:
user_vector_x = user_matrix_1[0]
cos_sim = []
    
for i in tqdm(range(len(article_20190222_20190314_matrix_1)),mininterval=10):
    cos_sim.append(cos_similarity(user_vector_x,article_20190222_20190314_matrix_1[i]))


100%|█████████████████████████████████████████████████████████████████████████| 15165/15165 [00:00<00:00, 36250.96it/s]


In [42]:
cos_sim[2207]

0.91977626

In [40]:
top_k = 100
cos_s = np.asarray(cos_sim)
cos_s.argsort()[-top_k -1 : -1][::-1].tolist()

[12246,
 2207,
 1210,
 12685,
 5499,
 9893,
 9299,
 7146,
 8050,
 11179,
 13911,
 10802,
 2615,
 13885,
 6367,
 13901,
 6134,
 1930,
 10826,
 2192,
 2324,
 6480,
 13086,
 2956,
 8431,
 7809,
 2999,
 4321,
 10867,
 7300,
 5093,
 4787,
 5236,
 535,
 14201,
 13383,
 12359,
 8927,
 176,
 11606,
 5326,
 1857,
 8068,
 12543,
 5687,
 213,
 15117,
 7145,
 1266,
 10658,
 14990,
 2809,
 12883,
 3190,
 9413,
 11651,
 9637,
 3988,
 2540,
 10345,
 14200,
 14919,
 7051,
 2139,
 8679,
 5386,
 197,
 14123,
 1526,
 7436,
 7608,
 6187,
 9141,
 4084,
 6497,
 264,
 3660,
 1537,
 6626,
 3460,
 541,
 7710,
 9903,
 13894,
 14629,
 5586,
 11763,
 1916,
 2745,
 4822,
 8538,
 6901,
 545,
 171,
 12614,
 6518,
 3238,
 6098,
 2122,
 14606]

# 아래는 메모리를 너무 많이 잡아먹어서 Pass

In [ ]:
tf.reset_default_graph() 
embeddings_article = tf.get_variable("artcle_matrix",initializer = article_total_matrix_1)
norm_article = tf.sqrt(tf.reduce_sum(tf.square(embeddings_article),1,keep_dims =True))
normalized_embeddings_article = embeddings_article / norm_article

embeddings_user = tf.get_variable("user_matrix",initializer = user_matrix_1)
norm_user = tf.sqrt(tf.reduce_sum(tf.square(embeddings_user),1,keep_dims =True))
normalized_embeddings_user = embeddings_user / norm_user


similarity = tf.matmul(normalized_embeddings_user, normalized_embeddings_article, transpose_b=True)

In [ ]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    sim = similarity.eval()